In [1]:
import os
import numpy as np
import cv2
import random
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.model_selection  import cross_val_score
import matplotlib.pyplot as plt

定义图片数据的路径和标签的路径

In [2]:
data_dir = 'C:\\Users\\86139\\face\\train'
label_dir = 'C:\\Users\\86139\\face\\train.txt'
data_list = os.listdir(data_dir)

img_path = []  # 用于存放jpg文件的路径    3993
labels = []  # 标签                  3993
images = []

打开图像文件夹读取图片的路径，保存图像路径在img_path

In [3]:
with open(label_dir, 'r') as f:
    for line in f.readlines():
        line = line.strip('\n')
        line = line.rstrip()
        words = line.split()  # words[0]代表xxxx.jpg; word[1]代表0或1

        path = os.path.join(data_dir, words[0])
        img_path.append(path)
        if words[1] == '1':
            labels.append(1)
        if words[1] == '0':
            labels.append(0)
        # labels.append(words[1])      # 写入标签

将灰度图像进行处理读取成字节信息，并转换成为数组

In [4]:
def read_img(img_path):
    for i in range(len(img_path)):
        if os.path.isfile(img_path[i]):
            images.append(cv2.imread(img_path[i], -1).flatten())


read_img(img_path)


images = np.array(images)
labels = np.array(labels)
print(images.shape)

(3993, 16384)


用PCA进行降维处理，提取特征值

In [5]:
pca = PCA(n_components=150)
pca = pca.fit(images)
images = pca.transform(images)
print(images.shape)
# 结果返回三列特征，也就是说3是最好的超参数取值

scaler = StandardScaler()
images = scaler.fit_transform(images)
print(images.shape)

(3993, 150)
(3993, 150)


进行训练集和测试集划分5：1

In [6]:
imagestrain,imagestest,labelstrain,labelstest=train_test_split(images,labels,test_size=0.2,random_state=42)
print(imagestrain.shape)

(3194, 150)


寻找KNN算法的最佳K值，并反馈训练最高得分

In [7]:
best_r = 0  # 最高准确率
best_k = 0  # 最佳k值

for i in range(1, 11):
#     print(i)
    knn_clf = KNeighborsClassifier(n_neighbors=i)  # 创建空分类器，默认超参数k值为5
    knn_clf.fit(imagestrain, labelstrain)  # 训练
    re = knn_clf.score(imagestrain, labelstrain) # 查看准确率
    print(re)
    
    if re > best_r:
        best_r = re
        best_k = i

print('最佳准确率为：', best_r)
print('最佳k值为：', best_k)



1.0
0.8315591734502191
0.8591108328115216
0.7370068879148404
0.7617407639323732
0.6803381340012523
0.6947401377582968
0.6421415153412648
0.65184721352536
0.6139636819035692
最佳准确率为： 1.0
最佳k值为： 1


进行KNN训练模型得到测试得分

In [8]:
knn_sex=KNeighborsClassifier(n_neighbors=1,p=1,metric = 'manhattan',weights='distance').fit(imagestrain,labelstrain)
print(knn_sex.score(imagestest,labelstest))#测试得分
print(knn_sex.score(imagestrain,labelstrain))#训练得分

0.7759699624530664
1.0
